In [127]:
import pandas as pd
import numpy as np
import shutil 
import os

import cv2 as cv
from IPython.display import clear_output

In [128]:
labels_list = os.listdir('runs/detect/predict/labels')
labels_list[:5]

['image_110.txt',
 'image_243.txt',
 'image_235.txt',
 'image_346.txt',
 'image_368.txt']

In [129]:
labels_dict = {
    'file_name': [],
    'xc_norm': [],
    'yc_norm': [],
    'width_norm': [],
    'height_norm': [],
    'confidence':[]
    }


In [130]:
labels_dir = 'runs/detect/predict/labels/'
for file in labels_list:

    with open(f'{labels_dir}{file}') as f:
        content = f.readline().strip().split(' ')

        labels_dict['file_name'].append(file.split('.')[0])
        labels_dict['xc_norm'].append(float(content[1]))
        labels_dict['yc_norm'].append(float(content[2]))
        labels_dict['width_norm'].append(float(content[3]))
        labels_dict['height_norm'].append(float(content[4]))
        labels_dict['confidence'].append(float(content[5]))


In [131]:
labels_df = pd.DataFrame(labels_dict)
labels_df.head()

,file_name,xc_norm,yc_norm,width_norm,height_norm,confidence
0,image_110,0.844696,0.498929,0.153846,0.279777,0.871451
1,image_243,0.512499,0.531153,0.966368,0.920114,0.636910
2,image_235,0.874789,0.659656,0.250423,0.497573,0.915062
3,image_346,0.202663,0.344388,0.404409,0.414154,0.898475
4,image_368,0.541119,0.493214,0.589268,0.533363,0.934718


In [132]:
labels_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71 entries, 0 to 70
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   file_name    71 non-null     object 
 1   xc_norm      71 non-null     float64
 2   yc_norm      71 non-null     float64
 3   width_norm   71 non-null     float64
 4   height_norm  71 non-null     float64
 5   confidence   71 non-null     float64
dtypes: float64(5), object(1)
memory usage: 3.5+ KB


In [133]:
img_list = [file for file in os.listdir('runs/detect/predict/') if file != 'labels']
img_list[:5]

['image_305.png',
 'image_431.jpeg',
 'image_375.png',
 'image_235.png',
 'image_311.png']

In [134]:
shapes = {
    'file_name':[],
    'img_h':[],
    'img_w':[]
    
}

for image in img_list:
    img = cv.imread('runs/detect/predict/' + image)
    img_name = image.split('/')[-1].split('.')[0]
    height, width, channels = img.shape 
    shapes['file_name'].append(img_name)
    shapes['img_w'].append(width)
    shapes['img_h'].append(height)

In [135]:
shapes_df = pd.DataFrame(shapes)
shapes_df.head()

,file_name,img_h,img_w
0,image_305,748,746
1,image_431,768,768
2,image_375,512,512
3,image_235,866,870
4,image_311,748,746


In [136]:
merged_df = labels_df.set_index('file_name').join(shapes_df.set_index('file_name'))
merged_df.head()


,xc_norm,yc_norm,width_norm,height_norm,confidence,img_h,img_w
file_name,,,,,,,
image_110,0.844696,0.498929,0.153846,0.279777,0.871451,768,768
image_243,0.512499,0.531153,0.966368,0.920114,0.636910,702,744
image_235,0.874789,0.659656,0.250423,0.497573,0.915062,866,870
image_346,0.202663,0.344388,0.404409,0.414154,0.898475,768,768
image_368,0.541119,0.493214,0.589268,0.533363,0.934718,768,768


In [137]:
merged_df.dtypes

xc_norm        float64
yc_norm        float64
width_norm     float64
height_norm    float64
confidence     float64
img_h            int64
img_w            int64
dtype: object

In [138]:

merged_df['x'] = merged_df['xc_norm']*merged_df['img_w']
merged_df['y'] = merged_df['yc_norm']*merged_df['img_h']
merged_df['w'] = merged_df['width_norm']*(merged_df['img_w'])
merged_df['h'] = merged_df['height_norm']*(merged_df['img_h'])

In [139]:
merged_df.head()

,xc_norm,yc_norm,width_norm,height_norm,confidence,img_h,img_w,x,y,w,h
file_name,,,,,,,,,,,
image_110,0.844696,0.498929,0.153846,0.279777,0.871451,768,768,648.726528,383.177472,118.153728,214.868736
image_243,0.512499,0.531153,0.966368,0.920114,0.636910,702,744,381.299256,372.869406,718.977792,645.920028
image_235,0.874789,0.659656,0.250423,0.497573,0.915062,866,870,761.066430,571.262096,217.868010,430.898218
image_346,0.202663,0.344388,0.404409,0.414154,0.898475,768,768,155.645184,264.489984,310.586112,318.070272
image_368,0.541119,0.493214,0.589268,0.533363,0.934718,768,768,415.579392,378.788352,452.557824,409.622784


In [140]:
merged_df['x1'] = merged_df['x']-(merged_df['w']/2)
merged_df['y1'] = merged_df['y']-(merged_df['h']/2)
merged_df['x2'] = merged_df['x']+(merged_df['w']/2)
merged_df['y2'] = merged_df['y']+(merged_df['h']/2)

In [141]:
merged_df.head()

,xc_norm,yc_norm,width_norm,height_norm,confidence,img_h,img_w,x,y,w,h,x1,y1,x2,y2
file_name,,,,,,,,,,,,,,,
image_110,0.844696,0.498929,0.153846,0.279777,0.871451,768,768,648.726528,383.177472,118.153728,214.868736,589.649664,275.743104,707.803392,490.611840
image_243,0.512499,0.531153,0.966368,0.920114,0.636910,702,744,381.299256,372.869406,718.977792,645.920028,21.810360,49.909392,740.788152,695.829420
image_235,0.874789,0.659656,0.250423,0.497573,0.915062,866,870,761.066430,571.262096,217.868010,430.898218,652.132425,355.812987,870.000435,786.711205
image_346,0.202663,0.344388,0.404409,0.414154,0.898475,768,768,155.645184,264.489984,310.586112,318.070272,0.352128,105.454848,310.938240,423.525120
image_368,0.541119,0.493214,0.589268,0.533363,0.934718,768,768,415.579392,378.788352,452.557824,409.622784,189.300480,173.976960,641.858304,583.599744


In [142]:
output_df = merged_df[['x1','y1','x2','y2','confidence']]
output_df

,x1,y1,x2,y2,confidence
file_name,,,,,
image_110,589.649664,275.743104,707.803392,490.611840,0.871451
image_243,21.810360,49.909392,740.788152,695.829420,0.636910
image_235,652.132425,355.812987,870.000435,786.711205,0.915062
image_346,0.352128,105.454848,310.938240,423.525120,0.898475
image_368,189.300480,173.976960,641.858304,583.599744,0.934718
...,...,...,...,...,...
image_101,151.945728,313.828224,478.487040,547.480704,0.937825
image_116,155.070720,212.402688,526.765824,768.000000,0.264547
image_94,113.911680,0.000000,767.999616,731.470848,0.508104


In [143]:
output_df.head()

,x1,y1,x2,y2,confidence
file_name,,,,,
image_110,589.649664,275.743104,707.803392,490.611840,0.871451
image_243,21.810360,49.909392,740.788152,695.829420,0.636910
image_235,652.132425,355.812987,870.000435,786.711205,0.915062
image_346,0.352128,105.454848,310.938240,423.525120,0.898475
image_368,189.300480,173.976960,641.858304,583.599744,0.934718


In [144]:
img_dict = {
    'file_name': img_list
}

In [145]:
all_test_images = pd.DataFrame(img_dict)
all_test_images.head()

,file_name
0,image_305.png
1,image_431.jpeg
2,image_375.png
3,image_235.png
4,image_311.png


In [146]:
all_test_images['to_list'] = all_test_images['file_name'].apply(lambda x: x.split('.')[:])
all_test_images['file_name'] = all_test_images['to_list'].apply(lambda x: x[0])
all_test_images['file_ext'] = all_test_images['to_list'].apply(lambda x: x[1])
all_test_images = all_test_images.drop('to_list',axis=1)
all_test_images.set_index('file_name')

,file_ext
file_name,
image_305,png
image_431,jpeg
image_375,png
image_235,png
image_311,png
...,...
image_69,jpeg
image_466,png
image_242,png


In [147]:
merged_outer = all_test_images.merge(output_df, on='file_name', how='outer')

In [148]:
merged_outer = merged_outer.fillna(0)

In [149]:
merged_outer.isna().sum()

file_name     0
file_ext      0
x1            0
y1            0
x2            0
y2            0
confidence    0
dtype: int64

In [150]:
merged_outer['x1'] = merged_outer['x1'].apply(lambda x: round(x))
merged_outer['x2'] = merged_outer['x2'].apply(lambda x: round(x))
merged_outer['y1'] = merged_outer['y1'].apply(lambda x: round(x))
merged_outer['y2'] = merged_outer['y2'].apply(lambda x: round(x))
clear_output()

In [151]:
merged_outer["file_name"] = merged_outer["file_name"] + '.' + merged_outer["file_ext"]

In [152]:
merged_outer = merged_outer.drop('file_ext',axis=1)

In [153]:
merged_outer

,file_name,x1,y1,x2,y2,confidence
0,image_305.png,0,0,0,0,0.000000
1,image_431.jpeg,263,428,737,768,0.756972
2,image_375.png,0,0,0,0,0.000000
3,image_235.png,652,356,870,787,0.915062
4,image_311.png,0,0,0,0,0.000000
...,...,...,...,...,...,...
144,image_69.jpeg,0,0,0,0,0.000000
145,image_466.png,30,308,510,663,0.969056
146,image_242.png,179,181,530,547,0.924250
147,image_30.jpeg,0,0,0,0,0.000000


In [154]:
merged_outer

,file_name,x1,y1,x2,y2,confidence
0,image_305.png,0,0,0,0,0.000000
1,image_431.jpeg,263,428,737,768,0.756972
2,image_375.png,0,0,0,0,0.000000
3,image_235.png,652,356,870,787,0.915062
4,image_311.png,0,0,0,0,0.000000
...,...,...,...,...,...,...
144,image_69.jpeg,0,0,0,0,0.000000
145,image_466.png,30,308,510,663,0.969056
146,image_242.png,179,181,530,547,0.924250
147,image_30.jpeg,0,0,0,0,0.000000


In [155]:
merged_outer.to_csv('../outputs/out.csv',index=False)